<a href="https://colab.research.google.com/github/Johanl001/Social-Media-Crisis-Management/blob/main/Crisis_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopy
!pip install folium

In [2]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
from folium.plugins import HeatMap
from collections import Counter

# Loading the dataset made in task 2
file_name = 'mental_health_posts_with_classification.csv'
data = pd.read_csv(file_name)


In [3]:
# Initializing geolocator
geolocator = Nominatim(user_agent="crisis_geo_locator")

def extract_location(text):
    if not isinstance(text, str):
        return None

    #Applying Simple regex to identify place names (e.g., "in Austin")
    match = re.search(r"in ([A-Za-z\s]+)", text)
    if match:
        location = match.group(1).strip()
        try:
            geo_location = geolocator.geocode(location, timeout=10)
            if geo_location:
                return (geo_location.latitude, geo_location.longitude)
        except GeocoderTimedOut:
            return None
    return None



In [4]:
#location extraction to content and title
data['location'] = data.apply(lambda row: extract_location(str(row['content']) + ' ' + str(row['title'])), axis=1)

# Filtering rows without location data
data_with_location = data.dropna(subset=['location'])

# Creating a list of locations for the heatmap
locations = data_with_location['location'].tolist()

# Creating a map centered on the mean coordinates
center_lat = sum(loc[0] for loc in locations) / len(locations)
center_lon = sum(loc[1] for loc in locations) / len(locations)
crisis_map = folium.Map(location=[center_lat, center_lon], zoom_start=4)

# Now, Adding a heatmap layer
HeatMap(locations).add_to(crisis_map)

# Finally, Saving the map
crisis_map.save("crisis_heatmap.html")

In [5]:
# Count occurrences of each location
location_counts = Counter(locations)

# top 5 locations with highest crisis discussions
top_5_locations = location_counts.most_common(5)

print("Top 5 locations with highest crisis discussions:")
for location, count in top_5_locations:
    print(f"Coordinates: {location}, Count: {count}")

Top 5 locations with highest crisis discussions:
Coordinates: (43.6606617, -79.3827952), Count: 10
Coordinates: (43.5459909, -80.2495008), Count: 8
Coordinates: (35.6145156, -88.4786605), Count: 7
Coordinates: (52.4297987, 13.4781392), Count: 5
Coordinates: (-1.2943004, 29.9170977), Count: 5


In [6]:
# markers for top 5 locations
for location, count in top_5_locations:
    folium.Marker(
        location=location,
        popup=f"Crisis discussions: {count}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(crisis_map)

# Saving the updated map with markers for top locations
crisis_map.save("crisis_heatmap_with_top_locations.html")

print("Heatmap and top locations visualization saved as 'crisis_heatmap_with_top_locations.html'")

Heatmap and top locations visualization saved as 'crisis_heatmap_with_top_locations.html'
